# Model Monitoring

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
!pip install evidently

  Obtaining dependency information for evidently from https://files.pythonhosted.org/packages/1e/53/83f20fd9677dbaa0f633ede6a1fe2738cc99fb6b9566a9f0123e7d0e6aa8/evidently-0.4.14-py3-none-any.whl.metadata
  Obtaining dependency information for fastapi>=0.100.0 from https://files.pythonhosted.org/packages/bf/97/60351307ab4502908d29f64f2801a36709a3f1888447bb328bc373d6ca0e/fastapi-0.109.2-py3-none-any.whl.metadata
  Obtaining dependency information for fastapi-restful>=0.5.0 from https://files.pythonhosted.org/packages/db/3d/e29e6685ab3f608ba445027bc9a6216fc0b576140938079f1c8290ea484f/fastapi_restful-0.5.0-py3-none-any.whl.metadata
  Obtaining dependency information for typing-inspect>=0.9.0 from https://files.pythonhosted.org/packages/65/f3/107a22063bf27bdccf2024833d3445f4eea42b2e598abfbd46f6a63b6cb0/typing_inspect-0.9.0-py3-none-any.whl.metadata
  Obtaining dependency information for uvicorn>=0.22.0 from https://files.pythonhosted.org/packages/c7/f3/29caa83f5795b20ed3aca357c648f3ae995ff6

In [ ]:
from evidently import ColumnMapping
from 